# Create a PISM domain from a polygon based on the BedMachine v5 grid

Calibration of flow parameters by sector. We could use this for auto-calibration.

In [ ]:
from pathlib import Path
import re

import cartopy
import cartopy.crs as ccrs
import geopandas as gp
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from pyDOE2 import lhs
from scipy.stats.distributions import randint, uniform
from shapely.geometry import Polygon
import xarray as xr

from pism_tutorials.domain import create_local_grid, get_bounds
from pism_tutorials.plotting import register_colormaps
from pism_tutorials.utils import merge_dicts, dict2str, sort_dict_by_key

xr.set_options(keep_attrs=True)

register_colormaps()

In [ ]:
# The name of the PISM Cloud S3 bucket
bucket_name = "pism-cloud-data"
! aws s3 sync --no-sign-request s3://pism-cloud-data/tutorial_files  tutorial_files

In [ ]:
# Coordinate Reference System 
crs = "EPSG:3413"

# the base resolution of BedMachine in meters
base_resolution: int = 150

# the resolutions that you want supported:
# 150, 300, 450, 600, 900, 1200, 1500, 1800, 2400, 3000, 3600, and 4500m
multipliers = [1, 2, 3, 4, 6, 8, 10, 12, 16, 20, 24, 30, 40, 60]
max_mult = multipliers[-1]

# buffer in m
buffer = 3e3

# Path to BedMachine file, open using xarray
ds = xr.open_dataset("tutorial_files/BedMachineGreenland-v5.nc")
ds = ds.rio.set_spatial_dims(x_dim="x", y_dim="y")
ds.rio.write_crs(crs, inplace=True)

x_main_bounds, y_main_bounds = get_bounds(ds, multipliers=multipliers)

In [ ]:
# Path to polygon file, open using GeoPandas
domains_gp = gp.read_file("tutorial_files/Greenland_Basins_PS_v1.4.2_w_shelves.gpkg").to_crs(crs).dropna(subset=["SUBREGION1"])
domains_gp.drop(domains_gp[domains_gp['SUBREGION1'].isin(["GIS"])].index, inplace=True)
print(domains_gp)

In [ ]:
# Basic example:
# We loop over all basins in the "basins" geopandas dataframe assuming that an "Name" attribute exists.
# First we add the buffer, then we extract the bounding box, and finally we calculate the domain and 
# save it as a netCDF file.

# This should be parallelized with Dask in the future.

cartopy_crs = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70, globe=None)
for m_id, domain in domains_gp.iterrows():
    name = domain["SUBREGION1"]
    print(f"Processing sector {name}")
    grid = create_local_grid(domain, ds, base_resolution=base_resolution, multipliers=multipliers, buffer=3e3)
    grid.attrs.update({"domain": name})
    grid.to_netcdf(f"domain_{name}.nc", engine="h5netcdf")
    x_point_list = [grid.x_bnds[0][0], grid.x_bnds[0][0], grid.x_bnds[0][1], grid.x_bnds[0][1], grid.x_bnds[0][0]]
    y_point_list = [grid.y_bnds[0][0], grid.y_bnds[0][1], grid.y_bnds[0][1], grid.y_bnds[0][0], grid.y_bnds[0][0]]
    polygon_geom = Polygon(zip(x_point_list, y_point_list))
    polygon = gp.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])
    polygon.to_file(f"domain_{name}.gpkg")



In [ ]:
n_samples = 1

flow_params_prior = {
            "basal_resistance.pseudo_plastic.q": uniform(0.25, 0.75),
            "basal_yield_stress.mohr_coulomb.till_effective_fraction_overburden": uniform(
                loc=0.01, scale=0.03
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_max": uniform(
                loc=40.0, scale=20.0
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_min": uniform(
                loc=5.0, scale=30.0
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_min": uniform(
                loc=-1000, scale=1000
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_max": uniform(
                loc=0, scale=1500
            ),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_min": uniform(loc=5.0, scale=30.0),
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_max": uniform(loc=40.0, scale=20.0),  
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_min": uniform(loc=-1000, scale=1000), 
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_max": uniform(loc=0, scale=1500), 
            "stress_balance.sia.enhancement_factor": uniform(loc=1.0, scale=3.0),
            "stress_balance.ssa.Glen_exponent": uniform(loc=2.75, scale=0.75),
            "stress_balance.sia.Glen_exponent": uniform(loc=1.0, scale=3.0),
        }

keys_prior = list(flow_params_prior.keys())
print("Prior Keys")
print("-" * 80)
print("\n".join([k for k in keys_prior]))


unif_sample = lhs(len(keys_prior), n_samples)
dist_sample = np.zeros_like(unif_sample)
for i, key in enumerate(keys_prior):
    dist_sample[:, i] = flow_params_prior[key].ppf(unif_sample[:, i])

uq_df = pd.DataFrame(dist_sample, columns=flow_params_prior.keys())

master_config_file = "tutorial_files/pism_config.nc"

def check_params(d: dict) -> None:
    print("\nChecking parameters")
    print("------------------------------------------------------------")
    with xr.open_dataset(master_config_file) as m_ds:
        for key in d:
            if hasattr(m_ds["pism_config"], key) is False:
                print(f"  - {key} not found in pism_config")
    print("------------------------------------------------------------\n")

flow_params_median = {
            "basal_resistance.pseudo_plastic.q": 0.7508221,
            "basal_yield_stress.mohr_coulomb.till_effective_fraction_overburden": 0.01845403,
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_max": 42.79528,
            "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_min": 7.193718,
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_max": 243.8239,
            "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_min": -369.6359,
            "stress_balance.sia.enhancement_factor": 2.608046,
            "stress_balance.ssa.Glen_exponent": 3.309718,
        }
median_df = pd.DataFrame([ list(flow_params_median.values())], columns=list(flow_params_median.keys()))
print(median_df)

In [ ]:
resolution = "3600m"  # a coarse resolution such that we can run the simulation on a desktop computer

output_dir = Path(f"testing_{resolution}")
output_dir.mkdir(parents=True, exist_ok=True)

extra_file = ""
start = "2015-01-01"
end = "2015-02-01"

input_params = {
    "bootstrap": "",
    "regional": "",
#    "i": "tutorial_files/BedMachineGreenland-v5.nc",
    "i": "tutorial_files/g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc",
    "input.regrid.file": "tutorial_files/g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc",
    "input.regrid.vars": "litho_temp,enthalpy,age,tillwat,bmelt,ice_area_specific_volume,thk"
}

grid_params = {
    "grid.dx": resolution,
    "grid.dy": resolution,
    "grid.Mz": 201,
    "grid.Lz": 4000,
    "grid.Mbz": 21,
    "grid.Lbz": 1000,
    "grid.file": None
}

time_params = {
    "time.start": start,
    "time.end": end,
    "time.calendar": "standard",
    "time_stepping.skip.enabled": "",
    "time_stepping.skip.max": 100
}

climate_params = {
    "surface.models": "given,forcing",
    "surface.given.file": "tutorial_files/MARv3.9_ERAI_climate_1978-2018_MEAN.nc",
#    "surface.force_to_thickness.file": "tutorial_files/BedMachineGreenland-v5.nc",
    "surface.force_to_thickness.file": "tutorial_files/g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc",
}

stress_balance = {
    "stress_balance": "ssa+sia",
    "stress_balance.calving_front_stress_bc": "",  
    "stress_balance.ice_free_thickness_standard": 5,
    "stress_balance.sia.bed_smoother.range": resolution[:-1], 
    "stress_balance.sia.enhancement_factor": 2.608046,
    "stress_balance.sia.flow_law": "gpbld",
    "stress_balance.ssa.Glen_exponent": 3.309718,
    "stress_balance.ssa.enhancement_factor": 1.0,
    "stress_balance.sia.max_diffusivity": 100000,
    "basal_resistance.pseudo_plastic.enabled": "yes",
    "basal_resistance.pseudo_plastic.q": 0.7508221,
    "basal_yield_stress.mohr_coulomb.till_effective_fraction_overburden": 0.01845403,
    "basal_yield_stress.mohr_coulomb.topg_to_phi.enabled": "yes",
    "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_max": 42.79528,
    "basal_yield_stress.mohr_coulomb.topg_to_phi.phi_min": 7.193718, 
    "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_max": 243.8239, 
    "basal_yield_stress.mohr_coulomb.topg_to_phi.topg_min": -369.6359, 
}

output_params = {
    "output.file": None,  # do not write a state file
    "output.format": "netcdf4_parallel",
    "output.extra.file": None,
    "output.extra.times": "yearly",
    "output.extra.vars": "velsurf_mag,usurf,thk,climatic_mass_balance,ice_surface_temp,mask,mass_fluxes,ice_mass_transport_across_grounding_line,ice_mass",
}

run_dict = merge_dicts(input_params, grid_params, time_params, stress_balance,climate_params, output_params)



In [ ]:
n = 8
for m_id, domain in domains_gp.iterrows():
    name = domain["SUBREGION1"]
    print(f"Running domain {name}")
    print("-" * 80)
    for s_id, sample in median_df.iterrows():
        print("\n")
        print("Parameters")
        print(sample)
        print("\n")
        
        run_dict.update(sample.to_dict())
        state_file = Path(output_dir) / f"state_g{resolution}_sector_{name}_id_{s_id}.nc"
        spatial_file = Path(output_dir) / f"spatial_g{resolution}_sector_{name}_id_{s_id}.nc"
        run_dict.update({"grid.file": f"domain_{name}.nc", "output.file": state_file, "output.extra.file": spatial_file})
        run_str = dict2str(sort_dict_by_key(run_dict))
        cmd = f"mpirun -np {n} pism " + run_str
        ! $cmd    
        # for s_id, sample in uq_df.iterrows():
        # run_dict.update(sample.to_dict())
        # state_file = Path(output_dir) / f"state_g{resolution}_sector_{name}_id_{s_id}.nc"
        # spatial_file = Path(output_dir) / f"spatial_g{resolution}_sector_{name}_id_{s_id}.nc"
        # run_dict.update({"grid.file": f"domain_{name}.nc", "output.file": state_file, "output.extra.file": spatial_file})
        # run_str = dict2str(sort_dict_by_key(run_dict))
        # cmd = f"mpirun -np {n} pism " + run_str
        # ! $cmd

In [ ]:
state_files = output_dir.glob(f"state_g{resolution}_*_id_0.nc")


time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)

main_domain = create_domain(x_main_bounds, y_main_bounds, resolution=int(resolution.split("m")[0]))


cartopy_crs = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70, globe=None)

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection=cartopy_crs)
states_clipped = []
for k, state_file in enumerate(state_files):
    domain_str = re.search("sector_(.+?)_", str(state_file)).group(1)
    domain_gp = domains_gp[domains_gp["SUBREGION1"].isin([domain_str])]
    add_colorbar = bool(k==0)
    print(f"Processing {domain_str} {state_file}")
    state_ds = xr.open_dataset(state_file, ).rio.set_spatial_dims(x_dim="x", y_dim="y")
    state_ds.rio.write_crs(crs, inplace=True)
    state_main_ds = state_ds.interp_like(main_domain)
    state_clipped_ds = state_main_ds.rio.clip(domain_gp.geometry, drop=False)
    state_clipped_ds = state_clipped_ds.expand_dims("sector")
    state_clipped_ds["sector"] = [domain_str]    
    print(state_clipped_ds.x[0])
    state_clipped_ds["velsurf_mag"].plot(ax=ax, vmin=10, vmax=1500, cmap="speed_colorblind", add_colorbar=add_colorbar)
    domain_gp.plot(ax=ax, facecolor="none", edgecolor="black", linewidth=0.25)
    states_clipped.append(state_clipped_ds)

In [ ]:
from collections import deque
dq = deque(states_clipped)
rotations = []
ms = []


for k in range(len(states_clipped)):
    rotations.append(list(dq))  # Convert deque to list and store
    m = xr.merge(list(dq), join="outer", compat='override')
    m=m.expand_dims("sector")
    m["sector"] = [k]
    ms.append(m)
    dq.rotate(-1)  

In [ ]:
xr.merge(ms, compat="override").sum(dim="sector")["velsurf_mag"].plot(vmin=10, vmax=1500, cmap="speed_colorblind", add_colorbar=add_colorbar)

In [ ]:
m = xr.concat(states_clipped, dim="sector")

In [ ]:
m["velsurf_mag"].mean(dim="sector").plot(vmin=10, vmax=1500, cmap="speed_colorblind", add_colorbar=add_colorbar)

In [ ]:
states_clipped[1].x

In [ ]:
ds = xr.Dataset({'x': ('x', np.linspace(0, 100, 91)), 'y': ('y', np.linspace(0, 200, 21))})
x_bounds, _ = get_bounds(ds)

fig, ax = plt.subplots(1, 1, figsize=(24, 8))
for k, m in enumerate(multipliers[:-5]):
    dr = m * base_resolution
    x = np.arange(x_bounds[0] + dr/2, x_bounds[1], dr)
    y = np.zeros_like(x) + k
    ax.plot(x, y, "x")
ax.grid()
ax.set_xticks(x)
ax.set_xlim(x_bounds[0], x_bounds[1])


In [ ]:
[state.x[0] for state in states_clipped]

In [ ]:
states_clipped[1]["velsurf_mag"].plot(vmin=10, vmax=1500, cmap="speed_colorblind", add_colorbar=add_colorbar)

In [ ]:
xr.merge([state["velsurf_mag"] for state in states_clipped], join="inner", compat="override")["velsurf_mag"].plot(vmin=10, vmax=1500, cmap="speed_colorblind", add_colorbar=add_colorbar)

In [ ]:
xr.merge([state["velsurf_mag"] for state in states_clipped], join="inner", compat="override")

In [ ]:
state_main_ds.rio.clip?

In [ ]:
xr.merge(ms)

In [ ]:
xr.open_dataset?

In [ ]:
x_main_bounds

In [ ]:
y_main_bounds